In [58]:
#from google.colab import files
import pandas as pd
import numpy as np
import calendar
# import warnings

import matplotlib.pyplot as plt

## Funciones

In [59]:
## FUNCION COMPLETITUD
## #######################
## #######################
## #######################

def function_completitud(df_x, variable, valor = 2, anio_filtro = '1901-01-01'):
    # Mostrando estadísticas importantes

    # --- Seleccionamos PLANTACION
    # ---
    df_bbdd_imputar = df_x
    plantacion_name = df_bbdd_imputar['PLANTACION'].unique()

    # Crear una nueva columna con el año y mes en el formato deseado
    df_bbdd_imputar['FECHA'] = pd.to_datetime(df_bbdd_imputar['FECHA'])

    # Crear un rango de fechas que cubra el período completo del DataFrame original
    start_date = df_bbdd_imputar['FECHA'].min().replace(day=1)
    end_date = (df_bbdd_imputar['FECHA'].max() + pd.offsets.MonthEnd(1)).replace(day=1) - pd.Timedelta(days=1)
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

    # Crear un DataFrame con todas las fechas
    df_all_dates = pd.DataFrame(all_dates, columns=['FECHA'])
    # Hacer un merge con el DataFrame original
    df_merged = pd.merge(df_all_dates, df_bbdd_imputar, on='FECHA', how='left')

    # Filtramos el año si es que fuera necesario
    # ---
    # Filtrar el DataFrame para considerar solo las fechas mayores a 2023-03-16
    fecha_limite = pd.Timestamp(anio_filtro)
    df_merged = df_merged[df_merged['FECHA'] > fecha_limite]

    # Crear las columnas variable_con_datos y variable_sin_datos
    df_merged['variable_con_datos'] = np.where(df_merged[variable].notnull() & (df_merged[variable] != 0), 1, 0)
    df_merged['variable_sin_datos'] = np.where(df_merged[variable].isnull() | (df_merged[variable] == 0), 1, 0)

    # Agrupar por mes y sumar las columnas
    df_merged['AÑO_MES'] = df_merged['FECHA'].dt.strftime('%Y-%m')
    df_monthly = df_merged.groupby('AÑO_MES').agg({'variable_con_datos': 'sum', 'variable_sin_datos': 'sum'}).reset_index()
    df_monthly['PLANTACION'] = plantacion_name[0]
    df_monthly['Total_dias'] = df_monthly['variable_con_datos'] + df_monthly['variable_sin_datos']
    df_monthly['% Completitud'] = df_monthly['variable_con_datos']/(df_monthly['variable_con_datos'] + df_monthly['variable_sin_datos'])

    # Agrupar por año
    df_merged['AÑO'] = df_merged['FECHA'].dt.strftime('%Y')
    df_year = df_merged.groupby('AÑO').agg({'variable_con_datos': 'sum', 'variable_sin_datos': 'sum'}).reset_index()
    df_year['PLANTACION'] = plantacion_name[0]
    df_year['Total_dias'] = df_year['variable_con_datos'] + df_year['variable_sin_datos']
    df_year['% Completitud'] = df_year['variable_con_datos']/(df_year['variable_con_datos'] + df_year['variable_sin_datos'])

    # Creando el indicador divergencia KL (Kullback-Leibler)
    # ------
    df_merged['incompleto'] = df_merged[variable]
    df_merged['completo'] = df_merged[variable]

    # Reemplazamos los ceros por NaN

    df_merged['completo'].replace(0, np.nan, inplace=True)
    df_merged['completo'].fillna(method='ffill', inplace=True) # Reemplazamos los valores NaN por el valor anterior (forward fill)

    # Calcular la moda de cada columna
    # moda_variable = df_merged['completo'].mode().iloc[0]

    # Verificar si la moda no está vacía antes de acceder al primer elemento
    if not df_merged['completo'].mode().empty:
        moda_variable = df_merged['completo'].mode().iloc[0]
        df_merged['completo'] = df_merged['completo'].fillna(moda_variable) # Reemplazamos los valores NaN por la moda
    else:
        # Si no hay moda, calcular el promedio
        moda_variable = df_merged['completo'].mean()
        df_merged['completo'] = df_merged['completo'].fillna(moda_variable) # Reemplazamos los valores NaN por el promedio


    df_merged['completo'] = df_merged['completo'].fillna(moda_variable) # Reemplazamos los valores NaN por el valor anterior (forward fill)

    # Calcular las distribuciones de la variable original y la imputada
    hist_original, bin_edges = np.histogram(df_merged['incompleto'].dropna(), bins=30, density=True)
    hist_imputado, _ = np.histogram(df_merged['completo'], bins=bin_edges, density=True)

    # Calcular la divergencia KL entre las dos distribuciones
    kl_divergence = entropy(hist_original, hist_imputado)


    # Guardamos el datframe - df_monthly
    # -----
    print( "## ################# ##")
    print(f'Divergencia KL: {kl_divergence}')
    print(f'Dataframe creados: df_year, df_month')
    print( "## ################# ##")

    # Especifica la ruta donde deseas guardar el excel
    ruta_missing_mensual =  '/PROYECTO_01_analitica_descriptiva_TEA/data/processed/missing_values/df_missing_values_month.xlsx'  # Cambia esta ruta a la que necesites
    df_monthly.to_excel(ruta_path + ruta_missing_mensual, index=False)
    print(f'Datos guardados en {ruta_path + ruta_missing_mensual}')

    # Guardamos el datframe - df_year
    # -----
    # Especifica la ruta donde deseas guardar el excel
    ruta_missing_anual =  '/PROYECTO_01_analitica_descriptiva_TEA/data/processed/missing_values/df_missing_values_year.xlsx'  # Cambia esta ruta a la que necesites
    df_year.to_excel(ruta_path + ruta_missing_anual, index=False)
    print(f'Datos guardados en {ruta_path + ruta_missing_anual}')

     # Mostrar el DataFrame correspondiente según el valor de `val`
    if valor == 1:
        print("DataFrame df_year:")
        print(df_year.head(10))
    else:
        print("DataFrame df_month:")
        print(df_monthly.head(10))



## FUNCION COMPLETITUD PENALIZADA
## #######################
## #######################
## #######################

def function_completitud_pen(df_x, variable, valor = 2, anio_filtro = '1901-01-01'):
    # Mostrando estadísticas importantes

    # --- Seleccionamos PLANTACION
    # ---
    df_bbdd_imputar = df_x
    plantacion_name = df_bbdd_imputar['PLANTACION'].unique()

    # Crear una nueva columna con el año y mes en el formato deseado
    df_bbdd_imputar['FECHA'] = pd.to_datetime(df_bbdd_imputar['FECHA'])

    # Crear un rango de fechas que cubra el período completo del DataFrame original
    start_date = df_bbdd_imputar['FECHA'].min().replace(day=1)
    end_date = (df_bbdd_imputar['FECHA'].max() + pd.offsets.MonthEnd(1)).replace(day=1) - pd.Timedelta(days=1)
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

    # Crear un DataFrame con todas las fechas
    df_all_dates = pd.DataFrame(all_dates, columns=['FECHA'])
    # Hacer un merge con el DataFrame original
    df_merged = pd.merge(df_all_dates, df_bbdd_imputar, on='FECHA', how='left')
    df_merged['FECHA'] = pd.to_datetime(df_merged['FECHA'])
    dim_start = df_merged.shape[0]

    # Filtrar filas donde el día de la semana no sea domingo (domingo es 6)
    df_merged = df_merged[df_merged['FECHA'].dt.weekday != 6]
    df_merged = df_merged[df_merged['FECHA'].dt.weekday != 5]

    dim_finish = df_merged.shape[0]

    dim_reduccion = dim_finish/dim_start

    # Filtramos el año si es que fuera necesario
    # ---
    # Filtrar el DataFrame para considerar solo las fechas mayores a 2023-03-16
    fecha_limite = pd.Timestamp(anio_filtro)
    df_merged = df_merged[df_merged['FECHA'] > fecha_limite]

    # Crear las columnas variable_con_datos y variable_sin_datos
    df_merged['variable_con_datos'] = np.where(df_merged[variable].notnull() & (df_merged[variable] != 0), 1, 0)
    df_merged['variable_sin_datos'] = np.where(df_merged[variable].isnull() | (df_merged[variable] == 0), 1, 0)

    sum_variable_con_datos = df_merged['variable_con_datos'].sum()
    sum_variable_sin_datos = df_merged['variable_sin_datos'].sum()
    pctj_completitud_total = sum_variable_con_datos/(sum_variable_con_datos + sum_variable_sin_datos)

    # Agrupar por mes y sumar las columnas
    df_merged['AÑO_MES'] = df_merged['FECHA'].dt.strftime('%Y-%m')
    df_monthly = df_merged.groupby('AÑO_MES').agg({'variable_con_datos': 'sum', 'variable_sin_datos': 'sum'}).reset_index()
    df_monthly['PLANTACION'] = plantacion_name[0]
    df_monthly['Total_dias'] = df_monthly['variable_con_datos'] + df_monthly['variable_sin_datos']
    df_monthly['% Completitud'] = df_monthly['variable_con_datos']/(df_monthly['variable_con_datos'] + df_monthly['variable_sin_datos'])

    # Agrupar por año
    df_merged['AÑO'] = df_merged['FECHA'].dt.strftime('%Y')
    df_year = df_merged.groupby('AÑO').agg({'variable_con_datos': 'sum', 'variable_sin_datos': 'sum'}).reset_index()
    df_year['PLANTACION'] = plantacion_name[0]
    df_year['Total_dias'] = df_year['variable_con_datos'] + df_year['variable_sin_datos']
    df_year['% Completitud'] = df_year['variable_con_datos']/(df_year['variable_con_datos'] + df_year['variable_sin_datos'])

    # Creando el indicador divergencia KL (Kullback-Leibler)
    # ------
    df_merged['incompleto'] = df_merged[variable]
    df_merged['completo'] = df_merged[variable]

    # Reemplazamos los ceros por NaN

    df_merged['completo'].replace(0, np.nan, inplace=True)
    df_merged['completo'].fillna(method='ffill', inplace=True) # Reemplazamos los valores NaN por el valor anterior (forward fill)

    # Calcular la moda de cada columna
    # moda_variable = df_merged['completo'].mode().iloc[0]

    # Verificar si la moda no está vacía antes de acceder al primer elemento
    if not df_merged['completo'].mode().empty:
        moda_variable = df_merged['completo'].mode().iloc[0]
        df_merged['completo'] = df_merged['completo'].fillna(moda_variable) # Reemplazamos los valores NaN por la moda
    else:
        # Si no hay moda, calcular el promedio
        moda_variable = df_merged['completo'].mean()
        df_merged['completo'] = df_merged['completo'].fillna(moda_variable) # Reemplazamos los valores NaN por el promedio


    df_merged['completo'] = df_merged['completo'].fillna(moda_variable) # Reemplazamos los valores NaN por el valor anterior (forward fill)

    # Calcular las distribuciones de la variable original y la imputada
    hist_original, bin_edges = np.histogram(df_merged['incompleto'].dropna(), bins=30, density=True)
    hist_imputado, _ = np.histogram(df_merged['completo'], bins=bin_edges, density=True)

    # Calcular la divergencia KL entre las dos distribuciones
    kl_divergence = entropy(hist_original, hist_imputado)


    # Guardamos el datframe - df_monthly
    # -----
    print( "## ################# ##")
    print(f'Divergencia KL: {kl_divergence}')
    print(f'Sin considerar dias sábados/domingos: {dim_reduccion}')
    print(f'Dataframe creados: df_year, df_month')
    print(f'pctj_completitud_total= {pctj_completitud_total}')
    print( "## ################# ##")

    # Especifica la ruta donde deseas guardar el excel
    ruta_missing_mensual =  '/PROYECTO_01_analitica_descriptiva_TEA/data/processed/missing_values/df_missing_values_month.xlsx'  # Cambia esta ruta a la que necesites
    df_monthly.to_excel(ruta_path + ruta_missing_mensual, index=False)
    print(f'Datos guardados en {ruta_path + ruta_missing_mensual}')

    # Guardamos el datframe - df_year
    # -----
    # Especifica la ruta donde deseas guardar el excel
    ruta_missing_anual =  '/PROYECTO_01_analitica_descriptiva_TEA/data/processed/missing_values/df_missing_values_year.xlsx'  # Cambia esta ruta a la que necesites
    df_year.to_excel(ruta_path + ruta_missing_anual, index=False)
    print(f'Datos guardados en {ruta_path + ruta_missing_anual}')

     # Mostrar el DataFrame correspondiente según el valor de `val`
    if valor == 1:
        print("DataFrame df_year:")
        print(df_year.head(10))
    else:
        print("DataFrame df_month:")
        print(df_monthly.head(10))






## FUNCION GRAFICADORA 1
## #######################
## #######################
## #######################

def function_tendencia_unica(df_x, valor,colors='green'):
    df_x = df_x.sort_values(by='FECHA')
    plantacion_name = df_x['PLANTACION'].unique()

    # Si solo te interesa modificar el primer valor (o un único valor) puedes hacer:
    if 'Shanuzi' in plantacion_name:
        plantacion_name = ['Shanuzi']
    else:
        plantacion_name = plantacion_name.tolist()
    
    plt.figure(figsize=(15, 5))
    plt.plot(df_x['FECHA'], df_x[valor], marker='o', markersize=2, linestyle='-', color=colors, label='Pesos')
    # plt.title(f'{plantacion_name[0]}: variable {valor} - a través del Tiempo ')
    plt.title(f'variable {valor} - a través del Tiempo ')
   
    plt.xlabel('Fecha')
    plt.ylabel('Pesos')
    plt.grid(True)
    plt.legend()
    #plt.xticks(rotation=45)

    fecha_min = df_x['FECHA'].min()
    fecha_max = df_x['FECHA'].max()
    print('Analisis de ', fecha_min, ' hasta ', fecha_max)
    
    # Mostrar la gráfica
    # print(df_TEA_full_tendencia[df_TEA_full_tendencia['RFF PROCESADO'] == 0].head())
    plt.tight_layout()
    plt.show()




## FUNCION CORRELACION CRUZADA
## #######################
## #######################
## #######################

def correlacion_cruzada(serie1, serie2):
    lags = np.arange(-len(serie1) + 1, len(serie1))
    corr = [serie1.corr(serie2.shift(lag)) for lag in lags]
    return lags, corr





## FUNCION SAVE EXCEL
## #######################
## #######################
## #######################

def function_save(df_x):
    # Especifica la ruta donde deseas guardar el CSV
    ruta_save =  '/PROYECTO_01_analitica_descriptiva_TEA/data/processed/__bbdd_revision__.xlsx'  # Cambia esta ruta a la que necesites
    df_x.to_excel(ruta_path + ruta_save, index=False)
    print(f'Datos guardados en {ruta_path + ruta_save}')



## FUNCION IMPUTACION OUTLIER
## #######################
## #######################
## #######################

def imputar_valores_extremos(df, variable, metodo='media'):
    """
    Imputa valores extremos en una variable de un DataFrame utilizando la media o la mediana.

    Parámetros:
    df (DataFrame): El DataFrame que contiene la variable a imputar.
    variable (str): El nombre de la variable que deseas imputar.
    metodo (str): La forma de imputación ('media' o 'mediana'). Por defecto es 'media'.

    Retorna:
    DataFrame: El DataFrame con la variable imputada.
    """
    if metodo not in ['media', 'mediana']:
        raise ValueError("El método debe ser 'media' o 'mediana'")

    # Calcular la media o la mediana
    if metodo == 'media':
        valor_imputacion = df[variable].mean()
    else:
        valor_imputacion = df[variable].median()

    # Identificar valores extremos (usando una regla de 3 veces la desviación estándar)
    limite_inferior = df[variable].mean() - 3 * df[variable].std()
    limite_superior = df[variable].mean() + 3 * df[variable].std()

    # Imputar valores extremos
    df[variable] = np.where(
        (df[variable] < limite_inferior) | (df[variable] > limite_superior),
        valor_imputacion,
        df[variable]
    )

    return df



## Carga de datos

In [60]:
# Cargamos el archivo Excel - RFF Procesado
# --------------

ruta_path = 'C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS'               # -> Casa
# ruta_path = 'C:/Users/usuario/Documents/DOCUMENTOS_CONSULTOR_5_BI'      # -> Trabajo

path_perdidas_aceite_pw = '/PROYECTO_01_analitica_descriptiva_TEA/data/external/bbdd_perdidas_aceite/704d248b-f0e9-441e-a80e-pw.xlsx'
path_perdidas_aceite_nh = '/PROYECTO_01_analitica_descriptiva_TEA/data/external/bbdd_perdidas_aceite/0b141767-ddcb-4cb5-ac23-nh.xlsx'
path_perdidas_aceite_sh = '/PROYECTO_01_analitica_descriptiva_TEA/data/external/bbdd_perdidas_aceite/ceceab95-d698-4ca2-a7dc-sh.xlsx'
path_perdidas_aceite_tl = '/PROYECTO_01_analitica_descriptiva_TEA/data/external/bbdd_perdidas_aceite/3d4df57c-0e6a-45ac-b670-tl.xlsx'


# Cargar la hoja 2 (indexada desde 0, por lo tanto la hoja 2 es 'sheet_name=1')
df_perdida_aceite_pw = pd.read_excel(ruta_path + path_perdidas_aceite_pw,sheet_name='Sheet1')
df_perdida_aceite_nh = pd.read_excel(ruta_path + path_perdidas_aceite_nh,sheet_name='Sheet1')
df_perdida_aceite_sh = pd.read_excel(ruta_path + path_perdidas_aceite_sh,sheet_name='Sheet1')
df_perdida_aceite_tl = pd.read_excel(ruta_path + path_perdidas_aceite_tl,sheet_name='Sheet1')

df_perdida_aceite_pw = df_perdida_aceite_pw[~df_perdida_aceite_pw[["AÑO", "MES", "DIA"]].isin(['Totales']).any(axis=1) & df_perdida_aceite_pw[["AÑO", "MES", "DIA"]].notna().all(axis=1)]
df_perdida_aceite_nh = df_perdida_aceite_nh[~df_perdida_aceite_nh[["AÑO", "MES", "DIA"]].isin(['Totales']).any(axis=1) & df_perdida_aceite_nh[["AÑO", "MES", "DIA"]].notna().all(axis=1)]
df_perdida_aceite_sh = df_perdida_aceite_sh[~df_perdida_aceite_sh[["AÑO", "MES", "DIA"]].isin(['Totales']).any(axis=1) & df_perdida_aceite_sh[["AÑO", "MES", "DIA"]].notna().all(axis=1)]
df_perdida_aceite_tl = df_perdida_aceite_tl[~df_perdida_aceite_tl[["AÑO", "MES", "DIA"]].isin(['Totales']).any(axis=1) & df_perdida_aceite_tl[["AÑO", "MES", "DIA"]].notna().all(axis=1)]

df_perdida_aceite_nh['PLANTACION'] = 'Nuevo Horizonte'
df_perdida_aceite_pw['PLANTACION'] = 'Palmawasi'
df_perdida_aceite_sh['PLANTACION'] = 'Shanuzi'
df_perdida_aceite_tl['PLANTACION'] = 'Tulumayo'

# Concatenar los DataFrames
df_perdida_aceite_full = pd.concat([df_perdida_aceite_nh, df_perdida_aceite_pw, df_perdida_aceite_sh, df_perdida_aceite_tl], ignore_index=True)

# Creamos variable fecha
df_perdida_aceite_full['FECHA'] = pd.to_datetime(df_perdida_aceite_full['DIA'])


print(df_perdida_aceite_full.dtypes)
df_perdida_aceite_full.head()


AÑO                                                int64
MES                                               object
DIA                                               object
RFF PROCESADO                                    float64
TM CONDENSADO                                    float64
TM ACEITE CONDENSADO                             float64
% PERDIDA ACEITE CONDENSADO                       object
TM ESCOBAJO SIN PICAR                            float64
TM ACEITE ESCOBAJO SIN PICAR                     float64
% PERDIDA ACEITE ESCOBAJO SIN PICAR               object
TM ESCOBAJO PICADO                               float64
TM ACEITE ESCOBAJO PICADO                        float64
% PERDIDA ACEITE ESCOBAJO PICADO                  object
% PERDIDA TOTAL ACEITE ESCOBAJO                   object
% RACIMOS DUROS                                   object
% RACIMOS MAL DESFRUTADO ANORMAL                  object
% RACIMO MAL DESFRUTADO                           object
FACTOR                         

,AÑO,MES,DIA,RFF PROCESADO,TM CONDENSADO,TM ACEITE CONDENSADO,% PERDIDA ACEITE CONDENSADO,TM ESCOBAJO SIN PICAR,TM ACEITE ESCOBAJO SIN PICAR,% PERDIDA ACEITE ESCOBAJO SIN PICAR,...,% PERDIDA ACEITE SOLIDO TD,TM ARENA,TM ACEITE EN ARENA,% PERDIDA ACEITE EN ARENA,TM NUEZ,TM ACEITE CASCARA,% PERDIDA ACEITE EN CASCARA,% PERDIDA TOTAL,PLANTACION,FECHA
0,2024,Enero,2024-01-01 00:00:00,0.00000,0.000000,0.000000,-,0.000000,0.000000,-,...,-,0.000000,0.000000,-,0.000000,0.000000,-,-,Nuevo Horizonte,2024-01-01
1,2024,Enero,2024-01-02 00:00:00,302.26061,19.042418,0.238901,0.00079,52.895607,2.382416,0.007882,...,0.002221,0.634747,0.057962,0.000192,21.158243,0.106710,0.000353,0.020257,Nuevo Horizonte,2024-01-02
2,2024,Enero,2024-01-03 00:00:00,338.41343,21.320046,0.258341,0.000763,59.222350,3.375230,0.009974,...,0.002029,0.710668,0.027868,0.000082,23.688940,0.165019,0.000488,0.026964,Nuevo Horizonte,2024-01-03
3,2024,Enero,2024-01-04 00:00:00,236.91708,14.925776,0.070097,0.000296,41.460489,2.362937,0.009974,...,0.001734,0.497526,0.033217,0.00014,16.584196,0.107534,0.000454,0.022926,Nuevo Horizonte,2024-01-04
4,2024,Enero,2024-01-05 00:00:00,264.46554,16.661329,0.075064,0.000284,46.281470,2.637697,0.009974,...,0.001735,0.555378,0.067796,0.000256,18.512588,0.124013,0.000469,0.024198,Nuevo Horizonte,2024-01-05


In [33]:
df_perdida_aceite_full[df_perdida_aceite_full['PLANTACION'] == 'Tulumayo'].tail()

,AÑO,MES,DIA,RFF PROCESADO,TM CONDENSADO,TM ACEITE CONDENSADO,% PERDIDA ACEITE CONDENSADO,TM ESCOBAJO SIN PICAR,TM ACEITE ESCOBAJO SIN PICAR,% PERDIDA ACEITE ESCOBAJO SIN PICAR,...,% PERDIDA ACEITE SOLIDO TD,TM ARENA,TM ACEITE EN ARENA,% PERDIDA ACEITE EN ARENA,TM NUEZ,TM ACEITE CASCARA,% PERDIDA ACEITE EN CASCARA,% PERDIDA TOTAL,PLANTACION,FECHA
1913,2024,Setiembre,2024-09-26 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,-,0.0,0.0,-,0.0,0.0,-,-,Tulumayo,2024-09-26
1914,2024,Setiembre,2024-09-27 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,-,0.0,0.0,-,0.0,0.0,-,-,Tulumayo,2024-09-27
1915,2024,Setiembre,2024-09-28 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,-,0.0,0.0,-,0.0,0.0,-,-,Tulumayo,2024-09-28
1916,2024,Setiembre,2024-09-29 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,-,0.0,0.0,-,0.0,0.0,-,-,Tulumayo,2024-09-29
1917,2024,Setiembre,2024-09-30 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,-,0.0,0.0,-,0.0,0.0,-,-,Tulumayo,2024-09-30


In [28]:
df_perdida_aceite_full.PLANTACION.unique()

array(['Nuevo Horizonte', 'Palmawasi', 'Shanuzi', 'Tulumayo'],
      dtype=object)

Añadiendo base adicional de perdidas

In [61]:
ruta_path = 'C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS'               # -> Casa
# ruta_path = 'C:/Users/usuario/Documents/DOCUMENTOS_CONSULTOR_5_BI'      # -> Trabajo

path_perdidas_aceite_todas_SEDES = '/PROYECTO_01_analitica_descriptiva_TEA/data/external/bbdd_perdidas_aceite/PERDIDAS HISTORICAS todas - SEDES.xlsx'


# Cargar la hoja 2 (indexada desde 0, por lo tanto la hoja 2 es 'sheet_name=1')
df_perdida_aceite_todas_SEDES = pd.read_excel(ruta_path + path_perdidas_aceite_todas_SEDES, sheet_name='news')

del df_perdida_aceite_todas_SEDES['RFF PROCESADO']

In [35]:
df_perdida_aceite_todas_SEDES.head()

,FECHA,PLANTACION,CONDENSADO,ESCOBAJO,FIBRA,CASCARA,AGUA,LODO,FRUTO ADHERIDO,ARENA
0,2022-01-01,Tulumayo,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,Tulumayo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-03,Tulumayo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-04,Tulumayo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-05,Tulumayo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_perdida_aceite_todas_SEDES.columns

Index(['FECHA', 'PLANTACION', 'CONDENSADO', 'ESCOBAJO', 'FIBRA', 'CASCARA',
       'AGUA', 'LODO', 'FRUTO ADHERIDO', 'ARENA'],
      dtype='object')

In [62]:
# Merge con df perdidas full
# ----
# ----

print("df_perdida_aceite_full inicial: ",df_perdida_aceite_full.shape)
print("df_perdida_aceite_todas_SEDES: ",df_perdida_aceite_todas_SEDES.shape)

# ----
df_perdida_aceite_full = pd.merge(df_perdida_aceite_full,
                                  df_perdida_aceite_todas_SEDES,
                                  on=['FECHA', 'PLANTACION'], 
                                  how='outer')

# ----
print("df_perdida_aceite_full final: ",df_perdida_aceite_full.shape)

df_perdida_aceite_full inicial:  (1918, 39)
df_perdida_aceite_todas_SEDES:  (4168, 10)
df_perdida_aceite_full final:  (4930, 47)


In [63]:
df_perdida_aceite_full.head()

,AÑO,MES,DIA,RFF PROCESADO,TM CONDENSADO,TM ACEITE CONDENSADO,% PERDIDA ACEITE CONDENSADO,TM ESCOBAJO SIN PICAR,TM ACEITE ESCOBAJO SIN PICAR,% PERDIDA ACEITE ESCOBAJO SIN PICAR,...,PLANTACION,FECHA,CONDENSADO,ESCOBAJO,FIBRA,CASCARA,AGUA,LODO,FRUTO ADHERIDO,ARENA
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Palmawasi,2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Shanuzi,2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Palmawasi,2021-01-02,0.806379,1.882027,5.850606,0.604603,2.667640,2.689077,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Shanuzi,2021-01-02,0.481212,6.905241,3.893720,0.300819,0.985154,1.082600,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Palmawasi,2021-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# Concatennando nuevas variables
# ----
df_perdida_aceite_full[['FECHA','PLANTACION','TM ACEITE FIBRA','FIBRA']].head()


,FECHA,PLANTACION,TM ACEITE FIBRA,FIBRA
0,2021-01-01,Palmawasi,NaN,NaN
1,2021-01-01,Shanuzi,NaN,NaN
2,2021-01-02,Palmawasi,NaN,5.850606
3,2021-01-02,Shanuzi,NaN,3.893720
4,2021-01-03,Palmawasi,NaN,NaN


In [66]:
# Crear la tercera columna basada en la condición

df_perdida_aceite_full['TM ACEITE ESCOBAJO'] = df_perdida_aceite_full['TM ACEITE ESCOBAJO SIN PICAR'] + df_perdida_aceite_full['TM ACEITE ESCOBAJO PICADO']
df_perdida_aceite_full['TM ACEITE ESCOBAJO_CMPTO'] = np.where((df_perdida_aceite_full['TM ACEITE ESCOBAJO'].isnull()| (df_perdida_aceite_full['TM ACEITE ESCOBAJO'] == 0)),df_perdida_aceite_full['ESCOBAJO'],df_perdida_aceite_full['TM ACEITE ESCOBAJO'])

df_perdida_aceite_full['TM ACEITE CONDENSADO_CMPTO'] = np.where((df_perdida_aceite_full['TM ACEITE CONDENSADO'].isnull()| (df_perdida_aceite_full['TM ACEITE CONDENSADO'] == 0)),df_perdida_aceite_full['CONDENSADO'],df_perdida_aceite_full['TM ACEITE CONDENSADO'])
df_perdida_aceite_full['TM ACEITE FIBRA_CMPTO'] = np.where((df_perdida_aceite_full['TM ACEITE FIBRA'].isnull()| (df_perdida_aceite_full['TM ACEITE FIBRA'] == 0)),df_perdida_aceite_full['FIBRA'],df_perdida_aceite_full['TM ACEITE FIBRA'])
df_perdida_aceite_full['TM ACEITE AGUA TD_CMPTO'] = np.where((df_perdida_aceite_full['TM ACEITE AGUA TD'].isnull()| (df_perdida_aceite_full['TM ACEITE AGUA TD'] == 0)),df_perdida_aceite_full['AGUA'],df_perdida_aceite_full['TM ACEITE AGUA TD'])
df_perdida_aceite_full['TM ACEITE EN ARENA_CMPTO'] = np.where((df_perdida_aceite_full['TM ACEITE EN ARENA'].isnull()| (df_perdida_aceite_full['TM ACEITE EN ARENA'] == 0)),df_perdida_aceite_full['ARENA'],df_perdida_aceite_full['TM ACEITE EN ARENA'])
df_perdida_aceite_full['TM ACEITE CASCARA_CMPTO'] = np.where((df_perdida_aceite_full['TM ACEITE CASCARA'].isnull()| (df_perdida_aceite_full['TM ACEITE CASCARA'] == 0)),df_perdida_aceite_full['CASCARA'],df_perdida_aceite_full['TM ACEITE CASCARA'])




In [54]:
# df_perdida_aceite_full[['FECHA','PLANTACION','TM ACEITE FIBRA','FIBRA','TM ACEITE FIBRA_CMPTO']].head()

df_perdida_aceite_full[['FECHA','PLANTACION','TM ACEITE ESCOBAJO',
                        'TM ACEITE ESCOBAJO SIN PICAR','TM ACEITE ESCOBAJO PICADO','TM ACEITE ESCOBAJO_CMPTO']].head()




,FECHA,PLANTACION,TM ACEITE ESCOBAJO,TM ACEITE ESCOBAJO SIN PICAR,TM ACEITE ESCOBAJO PICADO,TM ACEITE ESCOBAJO_CMPTO
0,2021-01-01,Palmawasi,NaN,NaN,NaN,NaN
1,2021-01-01,Shanuzi,NaN,NaN,NaN,NaN
2,2021-01-02,Palmawasi,NaN,NaN,NaN,1.882027
3,2021-01-02,Shanuzi,NaN,NaN,NaN,6.905241
4,2021-01-03,Palmawasi,NaN,NaN,NaN,NaN


In [42]:
df_perdida_aceite_full.shape

(4930, 47)

In [55]:
df_perdida_aceite_full.columns

Index(['AÑO', 'MES', 'DIA', 'RFF PROCESADO', 'TM CONDENSADO',
       'TM ACEITE CONDENSADO', '% PERDIDA ACEITE CONDENSADO',
       'TM ESCOBAJO SIN PICAR', 'TM ACEITE ESCOBAJO SIN PICAR',
       '% PERDIDA ACEITE ESCOBAJO SIN PICAR', 'TM ESCOBAJO PICADO',
       'TM ACEITE ESCOBAJO PICADO', '% PERDIDA ACEITE ESCOBAJO PICADO',
       '% PERDIDA TOTAL ACEITE ESCOBAJO', '% RACIMOS DUROS',
       '% RACIMOS MAL DESFRUTADO ANORMAL', '% RACIMO MAL DESFRUTADO', 'FACTOR',
       'TM ACEITE RACIMO MAL DESFRUTADO', '% EN PESO DE FRUTO ADHERIDO',
       '% PERDIDA ACEITE FRUTO ADHERIDO NORMAL', 'TM FIBRA', 'TM ACEITE FIBRA',
       '% PERDIDA ACEITE FIBRA', 'M3 AGUA TD', 'TM ACEITE AGUA TD',
       '% PERDIDA ACEITE AGUA TD', 'TM SOLIDOS TD', 'TM ACEITE SOLIDO TD',
       '% PERDIDA ACEITE SOLIDO TD', 'TM ARENA', 'TM ACEITE EN ARENA',
       '% PERDIDA ACEITE EN ARENA', 'TM NUEZ', 'TM ACEITE CASCARA',
       '% PERDIDA ACEITE EN CASCARA', '% PERDIDA TOTAL', 'PLANTACION', 'FECHA',
       'CONDENSA

In [67]:
function_save(df_perdida_aceite_full)

Datos guardados en C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS/PROYECTO_01_analitica_descriptiva_TEA/data/processed/__bbdd_revision__.xlsx


### Guardando datos de PERDIDAS EN ACEITE

In [68]:
# Guardamos la data en processed
# -----
# Especifica la ruta donde deseas guardar el CSV
ruta_perdida_aceite_full =  '/PROYECTO_01_analitica_descriptiva_TEA/data/processed/df_perdida_aceite_full.csv'  # Cambia esta ruta a la que necesites
# Guardar el DataFrame en formato CSV
df_perdida_aceite_full.to_csv(ruta_path + ruta_perdida_aceite_full, index=False)
print(f'Datos guardados en {ruta_path + ruta_perdida_aceite_full}')

df_perdida_aceite_full.tail(10)

Datos guardados en C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS/PROYECTO_01_analitica_descriptiva_TEA/data/processed/df_perdida_aceite_full.csv


,AÑO,MES,DIA,RFF PROCESADO,TM CONDENSADO,TM ACEITE CONDENSADO,% PERDIDA ACEITE CONDENSADO,TM ESCOBAJO SIN PICAR,TM ACEITE ESCOBAJO SIN PICAR,% PERDIDA ACEITE ESCOBAJO SIN PICAR,...,LODO,FRUTO ADHERIDO,ARENA,TM ACEITE ESCOBAJO,TM ACEITE ESCOBAJO_CMPTO,TM ACEITE CONDENSADO_CMPTO,TM ACEITE FIBRA_CMPTO,TM ACEITE AGUA TD_CMPTO,TM ACEITE EN ARENA_CMPTO,TM ACEITE CASCARA_CMPTO
4920,2024.0,Diciembre,2024-12-22 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4921,2024.0,Diciembre,2024-12-23 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4922,2024.0,Diciembre,2024-12-24 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4923,2024.0,Diciembre,2024-12-25 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4924,2024.0,Diciembre,2024-12-26 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4925,2024.0,Diciembre,2024-12-27 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4926,2024.0,Diciembre,2024-12-28 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4927,2024.0,Diciembre,2024-12-29 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4928,2024.0,Diciembre,2024-12-30 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4929,2024.0,Diciembre,2024-12-31 00:00:00,0.0,0.0,0.0,-,0.0,0.0,-,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
